# APIs

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Import

In [9]:
data = pd.read_csv('adult.data', index_col=False, names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
 'marital-status', 'occupation', 'relationship', 'race', 'gender',
 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
 'income'])

In [10]:
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
 'occupation', 'income']]

In [11]:
data.head()

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


In [12]:
data.shape

(32561, 7)